<a href="https://colab.research.google.com/github/karl0212/PyNite/blob/master/Confusion_Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow-gpu==2.0.0-beta1

     |████████████████████████████████| 348.9MB 60kB/s 
     |████████████████████████████████| 3.1MB 38.6MB/s 
     |████████████████████████████████| 501kB 47.8MB/s 


In [1]:
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import tensorflow as tf

import numpy as np

import seaborn as sns

import pandas as pd

In [2]:
tf.__version__

'2.8.2'

In [ ]:
%load_ext tensorboard

In [ ]:
logdir='log'

In [3]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

train_images, test_images = train_images / 255.0, test_images / 255.0

classes=[0,1,2,3,4,5,6,7,8,9]

11501568/11490434 [==============================] - 0s 0us/step


In [4]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))


In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [5]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x=train_images, 
            y=train_labels, 
            epochs=5, 
            validation_data=(test_images, test_labels))

Epoch 1/5
1875/1875 [==============================] - 72s 38ms/step - loss: 0.1554 - accuracy: 0.9521 - val_loss: 0.0456 - val_accuracy: 0.9843
Epoch 2/5
1875/1875 [==============================] - 63s 33ms/step - loss: 0.0470 - accuracy: 0.9857 - val_loss: 0.0486 - val_accuracy: 0.9846
Epoch 3/5
1875/1875 [==============================] - 72s 38ms/step - loss: 0.0340 - accuracy: 0.9893 - val_loss: 0.0366 - val_accuracy: 0.9867
Epoch 4/5
1875/1875 [==============================] - 65s 35ms/step - loss: 0.0269 - accuracy: 0.9915 - val_loss: 0.0340 - val_accuracy: 0.9899
Epoch 5/5
1875/1875 [==============================] - 63s 34ms/step - loss: 0.0204 - accuracy: 0.9939 - val_loss: 0.0314 - val_accuracy: 0.9911


In [9]:
y_true=test_labels
y_pred=model.predict(test_images)
print (y_true)
print (np.shape(y_true))



[7 2 1 ... 4 5 6]
(10000,)


In [15]:
classes=[0,1,2,3,4,5,6,7,8,9]
print (y_pred)
print (np.shape(y_pred))

[[3.1377234e-11 9.1689108e-11 2.3086866e-09 ... 1.0000000e+00
  1.2806828e-10 2.9439232e-10]
 [5.1363433e-08 1.3450497e-06 9.9999869e-01 ... 4.8192023e-10
  8.5369857e-11 1.2692714e-13]
 [1.5300159e-08 9.9999106e-01 1.5265966e-06 ... 1.3224350e-06
  1.6067984e-06 5.1983164e-08]
 ...
 [2.5865882e-16 2.8854590e-09 1.7087285e-11 ... 5.9901933e-10
  1.8637709e-09 6.2461375e-10]
 [9.0112717e-10 1.3971648e-07 5.0173581e-09 ... 5.1789051e-10
  1.3275736e-05 2.5001992e-09]
 [7.5197176e-10 4.4708674e-08 4.3542919e-08 ... 1.1040494e-12
  1.6692493e-08 1.2922185e-11]]
(10000, 10)


In [12]:
con_mat = tf.math.confusion_matrix(labels=y_true, predictions=y_pred).numpy()
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

con_mat_df = pd.DataFrame(con_mat_norm,
                     index = classes, 
                     columns = classes)

figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

InvalidArgumentError: ignored

In [ ]:
model1 = models.Sequential()
model1.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model1.add(layers.MaxPooling2D((2, 2)))
model1.add(layers.Conv2D(64, (3, 3), activation='relu'))
model1.add(layers.MaxPooling2D((2, 2)))
model1.add(layers.Conv2D(64, (3, 3), activation='relu'))

model1.add(layers.Flatten())
model1.add(layers.Dense(64, activation='relu'))
model1.add(layers.Dense(10, activation='softmax'))

model1.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
file_writer = tf.summary.create_file_writer(logdir + '/cm')

def log_confusion_matrix(epoch, logs):
  # Use the model to predict the values from the validation dataset.
  test_pred = model1.predict_classes(test_images)

  con_mat = tf.math.confusion_matrix(labels=test_labels, predictions=test_pred).numpy()
  con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

  con_mat_df = pd.DataFrame(con_mat_norm,
                     index = classes, 
                     columns = classes)

  figure = plt.figure(figsize=(8, 8))
  sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  
  buf = io.BytesIO()
  plt.savefig(buf, format='png')

  plt.close(figure)
  buf.seek(0)
  image = tf.image.decode_png(buf.getvalue(), channels=4)

  image = tf.expand_dims(image, 0)
  
  # Log the confusion matrix as an image summary.
  with file_writer.as_default():
    tf.summary.image("Confusion Matrix", image, step=epoch)

    
logdir='logs/images'

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

cm_callback = keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
model1.fit(
    train_images,
    train_labels,
    epochs=5,
    verbose=0, 
    callbacks=[tensorboard_callback, cm_callback],
    validation_data=(test_images, test_labels),
)

In [ ]:
# Start TensorBoard.
%tensorboard --logdir logs/images